In [1]:
import sys
import os
import h5py
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import warnings
warnings.filterwarnings("ignore")

# Get the absolute path to the src directory
sys.path.append(os.path.abspath("../src"))
from compute_flops import compute_flops


- Start with 1000 time series
- Split immediately → 700 for training, 300 for evaluation
- Tokenise the 700 training series into sequences
- Train the model on this
- Evaluate the model on the validation/test split (the 300)

## 3A

In [ ]:
from compute_flops import compute_flops

compute_flops(
    data_path="../lotka_volterra_data.h5",  # your dataset
    input_fraction=0.7,                     # 70% input, 30% to predict
    lora_rank=0,                            # no LoRA => untrained
    batch_size=1,                           # batch size irrelevant here
    training_steps=0,                       # no training
    flop_budget=1e17,                       # your FLOP budget
    train_series_count=0,                   # not needed for untrained
    eval_series_count=300,                 # evaluate all 300 test series
    context_length=512                     # fixed context length
)



📏 Context length used for FLOP estimation: 512 tokens

 💡 Total Training FLOPs: 0.00e+00
 🧪 Total Evaluation FLOPs: 1.69e+14
 🔢 Total Combined FLOPs: 1.69e+14
 🎯 Percentage of FLOP budget used: 0.16879%


In [4]:
from compute_flops import compute_flops

compute_flops(
    data_path="../lotka_volterra_data.h5",  # or your full path
    input_fraction=0.7,                     # 70% of each series used as input
    lora_rank=4,                            # default LoRA rank
    batch_size=4,                           # reasonable batch size
    training_steps=1000,                    # only doing 1000 steps for now
    flop_budget=1e17,                       # your FLOP budget
    train_series_count=700,                 # 70% of 1000 series
    eval_series_count=300,                  # evaluate on remaining 30%
    context_length=512                      # fixed context length
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



📏 Context length used for FLOP estimation: 512 tokens

 💡 Total Training FLOPs: 6.76e+15
 🧪 Total Evaluation FLOPs: 1.69e+14
 🔢 Total Combined FLOPs: 6.92e+15
 🎯 Percentage of FLOP budget used: 6.92494%


## 3B

#### LoRA = 2 scenarios

In [11]:
compute_flops(
    data_path="../lotka_volterra_data.h5",
    input_fraction=0.7,
    lora_rank=2,
    batch_size=4,
    training_steps=1000,
    flop_budget=1e17,
    train_series_count=700,
    eval_series_count=300,
    context_length=512
)

compute_flops(
    data_path="../lotka_volterra_data.h5",
    input_fraction=0.7,
    lora_rank=4,
    batch_size=4,
    training_steps=1000,
    flop_budget=1e17,
    train_series_count=700,
    eval_series_count=300,
    context_length=512
)

compute_flops(
    data_path="../lotka_volterra_data.h5",
    input_fraction=0.7,
    lora_rank=8,
    batch_size=4,
    training_steps=1000,
    flop_budget=1e17,
    train_series_count=700,
    eval_series_count=300,
    context_length=512
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



📏 Context length used for FLOP estimation: 512 tokens

 💡 Total Training FLOPs: 6.75e+15
 🧪 Total Evaluation FLOPs: 1.69e+14
 🔢 Total Combined FLOPs: 6.92e+15
 🎯 Percentage of FLOP budget used: 6.92277%


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



📏 Context length used for FLOP estimation: 512 tokens

 💡 Total Training FLOPs: 6.76e+15
 🧪 Total Evaluation FLOPs: 1.69e+14
 🔢 Total Combined FLOPs: 6.92e+15
 🎯 Percentage of FLOP budget used: 6.92494%


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



📏 Context length used for FLOP estimation: 512 tokens

 💡 Total Training FLOPs: 6.76e+15
 🧪 Total Evaluation FLOPs: 1.69e+14
 🔢 Total Combined FLOPs: 6.93e+15
 🎯 Percentage of FLOP budget used: 6.92927%


The above should all be multipled by 3 since I will run 9 tests

---

Following this I will choose the best hyperparameters to vary context length from 128, 512, 768